## LangChain 라이브러리에서 청킹(Chunking)을 지원해주는 함수
- CharacterTextSplitter
    - 텍스트를 지정된 기준으로 나눠주는 기본적인 함수
    - 구분자는 디폴트로 "\n\n"을 사용하지만 변경 가능
    - 구분자가 단일 문자이기 때문에, 연속된 텍스트의 의미를 충분히 고려하지 못할 수 있음
- RecursiveCharacterTextSplitter
    - 텍스트를 여러 단계에 걸쳐 나누어 주는 함수 (청킹에서 가장 많이 활용 됨)
    - 리스트를 통해 구분자를 여러 개 사용하여 세분화 된 청크를 만들 수 있음 (구분자의 순서가 중요)
    - 처음에는 "\n\n" 으로 분리하고 청크가 여전히 크다면 "\n", 그래도 크다면 " "과 같은 식으로 지정된 크기 제한 내에 들어올 때까지 계층적으로 분할 진행

In [1]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [2]:
chunk_size = 20 # 하나의 청크에 대한 길이를 문자 20개로 설정
chunk_overlap = 0

c_splitter = CharacterTextSplitter(
    # 청크 크기 지정
    # 1. 모델이 처리 가능한 최대 토큰 수를 넘지 않게 설정 (사용자 질의 + 모델 응답)
    # 2. 의미있는 문맥을 유지하기 위해서 일반적인 문서에는 보통 200 ~ 1500사이로 지정
    # 3. 데이터(문서)의 종류에 따라 달라짐, 기술 문서나 연구 논문은 청크가 큰 게 좋으며, 뉴스 기나사 소설은 작은게 좋을 수 있음
    chunk_size = chunk_size,
    # 청크 간 겹치는 부분의 크기 지정 (청킹 후, 사이에 있는 텍스트가 소실되는 경우 지정해주면 됨)
    chunk_overlap = chunk_overlap
)

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)

In [3]:
txt1 = "청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요?"
txt2 = "청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요? 결과를 확인해 봅시다."
txt3 = """
청킹을 위한 예제 데이터입니다.
어떻게 쪼개지는지 알아볼까요?

결과를 확인해 봅시다.
"""

- CharacterTextSplitter

In [4]:
# split_text() : 텍스트 데이터 청킹 실행 함수
c_splitter.split_text(txt1)

['청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요?']

In [5]:
c_splitter.split_text(txt2)

['청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요? 결과를 확인해 봅시다.']

In [6]:
c_splitter.split_text(txt3)
# 발생한 경고 메시지는 지정된 20자 크기보다 더 큰 청크를 생성했다는 뜻으로 디폴트로 지정된 구분 기준이 "\n\n"이 
# 텍스트 내에서 충분히 나타나지 않아서 20자 이내로 나눌 수 없는 경우에 발생함

Created a chunk of size 35, which is longer than the specified 20


['청킹을 위한 예제 데이터입니다.\n어떻게 쪼개지는지 알아볼까요?', '결과를 확인해 봅시다.']

- RecursiveCharacterTextSplitter

In [7]:
r_splitter.split_text(txt1)

['청킹을 위한 예제 데이터입니다.', '어떻게 쪼개지는지 알아볼까요?']

In [8]:
r_splitter.split_text(txt2)

['청킹을 위한 예제 데이터입니다.', '어떻게 쪼개지는지 알아볼까요?', '결과를 확인해 봅시다.']

In [9]:
r_splitter.split_text(txt3)

['청킹을 위한 예제 데이터입니다.', '어떻게 쪼개지는지 알아볼까요?', '결과를 확인해 봅시다.']

In [10]:
deep_deep_sleep = """
I want a deep, deep sleep tonight
But my phone, beep-beep, crying
I don't wanna answer my phone
I knocked on the dream's door again

I walk in the street, arrived in the moon
Cars on the sky, planes rushing by
Cotton candy flavor fog (hey, hey, hey, hey)
Strangely odd, not unusual at all
And I'm just deep, deep sleeping
PJs with a sweet dream
No need for questions, no need for supervisions
Now I got no problems 'til I gotta wake up, hm-hm
"""
len(deep_deep_sleep)

444

In [11]:
c_splitter = CharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    separator = "\n\n" # 청크 간 구분에 대한 기준 (사용자가 직접 설정 가능)
)
c_splitter.split_text(deep_deep_sleep)

Created a chunk of size 132, which is longer than the specified 20


["I want a deep, deep sleep tonight\nBut my phone, beep-beep, crying\nI don't wanna answer my phone\nI knocked on the dream's door again",
 "I walk in the street, arrived in the moon\nCars on the sky, planes rushing by\nCotton candy flavor fog (hey, hey, hey, hey)\nStrangely odd, not unusual at all\nAnd I'm just deep, deep sleeping\nPJs with a sweet dream\nNo need for questions, no need for supervisions\nNow I got no problems 'til I gotta wake up, hm-hm"]

In [12]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    separators = ["\n\n", "\n", " "] # 리스트 내에 여러 개 지정 가능
)
r_splitter.split_text(deep_deep_sleep)

["I want a deep, deep sleep tonight\nBut my phone, beep-beep, crying\nI don't wanna answer my phone",
 "I knocked on the dream's door again",
 'I walk in the street, arrived in the moon\nCars on the sky, planes rushing by',
 'Cotton candy flavor fog (hey, hey, hey, hey)\nStrangely odd, not unusual at all',
 "And I'm just deep, deep sleeping\nPJs with a sweet dream",
 "No need for questions, no need for supervisions\nNow I got no problems 'til I gotta wake up, hm-hm"]

- CharacterTextSplitter를 사용한 결과가 만족스럽지 못하다면 더 작은 단위의 구분자를 지정하여 해결 하거나
- RecursiveCharacterTextSplitter를 사용하여 계층적으로 분할을 적용해 볼 수 있음

### 웹 페이지에 접근해 텍스트 데이터 가져오기

In [13]:
# WebBaseLoader : 특정 웹 페이지의 내용을 로드하고 파싱하기 위해 설계된 클래스 (내부적으로 BeautifulSoup 사용)
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
loader = WebBaseLoader("https://n.news.naver.com/mnews/article/009/0005331043")

# 브라우저 우회 접속
loader.requests_kwargs = {"headers" : {"User-Agent" : "Mozilla/5.0"}}
loader.encoding = "utf-8"

docs = loader.load()
docs

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n“외출 망설여지네”…천둥·번개 동반 수도권 최대 100㎜ ‘물폭탄’\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문 바로가기\n\n\n\n\n\n\nNAVER\n\n뉴스\n\n\n연예\n\n\n\n\n스포츠\n\n\n\n\n날씨\n\n\n\n\n프리미엄\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n언론사별\n\n\n정치\n\n\n경제\n\n\n사회\n\n\n생활/문화\n\n\nIT/과학\n\n\n세계\n\n\n랭킹\n\n\n신문보기\n\n\n오피니언\n\n\nTV\n\n\n팩트체크\n\n\n알고리즘 안내\n\n\n정정보도 모음\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n구독\n\n매일경제 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n매일경제 언론사 구독 해지되었습니다.\n\n\n\n\nPICK\n안내\n\n\n언론사가 주요기사로선정한 기사입니다.\n언론사별 바로가기\n닫기\n\n\n\n\n“외출 망설여지네”…천둥·번개 동반 수도권 최대 100㎜ ‘물폭탄’\n\n\n\n\n입력2024.07.08. 오전 11:16\n\n기사원문\n \n\n\n\n\n이가람 기자\n\n\n\n\n\n\n\n\n\n\n이가람 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 

In [15]:
docs[0].page_content[:500]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n“외출 망설여지네”…천둥·번개 동반 수도권 최대 100㎜ ‘물폭탄’\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문 바로가기\n\n\n\n\n\n\nNAVER\n\n뉴스\n\n\n연예\n\n\n\n\n스포츠\n\n\n\n\n날씨\n\n\n\n\n프리미엄\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n언론사별\n\n\n정치\n\n\n경제\n\n\n사회\n\n\n생활/문화\n\n\nIT/과학\n\n\n세계\n\n\n랭킹\n\n\n신문보기\n\n\n오피니언\n\n\nTV\n\n\n팩트체크\n\n\n알고리즘 안내\n\n\n정정보도 모음\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n구독\n\n매일경제 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n매일경제 언론사 구독 해지되었습니다.\n\n\n\n\nPICK\n안내\n\n\n언론사가 주요기사로선정한 기사입니다.\n언론사별 바로가기\n닫기\n\n\n\n\n“외출 망설여지네”…천둥·번개 동반 수도권 최대 100㎜ ‘물폭탄’\n\n\n\n\n입력2024.07.08. 오전 11:16\n\n기사원문\n \n\n\n\n\n이가람 기자\n\n\n\n\n\n\n\n\n\n\n이가람 기자\n\n구독\n구독중\n\n\n\n\n구'

In [16]:
print(docs[0].page_content[:500])




















“외출 망설여지네”…천둥·번개 동반 수도권 최대 100㎜ ‘물폭탄’













본문 바로가기






NAVER

뉴스


연예




스포츠




날씨




프리미엄










검색










언론사별


정치


경제


사회


생활/문화


IT/과학


세계


랭킹


신문보기


오피니언


TV


팩트체크


알고리즘 안내


정정보도 모음



















구독

매일경제 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.
보러가기


매일경제 언론사 구독 해지되었습니다.




PICK
안내


언론사가 주요기사로선정한 기사입니다.
언론사별 바로가기
닫기




“외출 망설여지네”…천둥·번개 동반 수도권 최대 100㎜ ‘물폭탄’




입력2024.07.08. 오전 11:16

기사원문
 




이가람 기자










이가람 기자

구독
구독중




구


- 문자열 함수 replace 함수 활용

In [17]:
docs[0].page_content[:500].replace("\n", "")

'“외출 망설여지네”…천둥·번개 동반 수도권 최대 100㎜ ‘물폭탄’본문 바로가기NAVER뉴스연예스포츠날씨프리미엄검색언론사별정치경제사회생활/문화IT/과학세계랭킹신문보기오피니언TV팩트체크알고리즘 안내정정보도 모음구독매일경제 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.보러가기매일경제 언론사 구독 해지되었습니다.PICK안내언론사가 주요기사로선정한 기사입니다.언론사별 바로가기닫기“외출 망설여지네”…천둥·번개 동반 수도권 최대 100㎜ ‘물폭탄’입력2024.07.08. 오전 11:16기사원문 이가람 기자이가람 기자구독구독중구'

### 💡 문자열 정규표현식 활용
- 정규표현식 검색 결과가 개별 문자열로 출력되기 때문에 필요에 따라 이를 모두 합쳐서 하나의 문자열로 활용해야 함

In [18]:
import re

In [19]:
# 가-힣 : 한글 문자열 전체
# a-zA-Z : 영문 문자열 (대, 소문자 전체)
# 0-9_ : 숫자 및 언더바
# [] : 대괄호
# + : 연속된 문자열 반환 (띄어쓰기)!
re.findall("[가-힣a-zA-Z0-9_]+", docs[0].page_content[:500]) # 문자만 출력하는 정규 표현식

['외출',
 '망설여지네',
 '천둥',
 '번개',
 '동반',
 '수도권',
 '최대',
 '100',
 '물폭탄',
 '본문',
 '바로가기',
 'NAVER',
 '뉴스',
 '연예',
 '스포츠',
 '날씨',
 '프리미엄',
 '검색',
 '언론사별',
 '정치',
 '경제',
 '사회',
 '생활',
 '문화',
 'IT',
 '과학',
 '세계',
 '랭킹',
 '신문보기',
 '오피니언',
 'TV',
 '팩트체크',
 '알고리즘',
 '안내',
 '정정보도',
 '모음',
 '구독',
 '매일경제',
 '언론사',
 '구독되었습니다',
 '메인',
 '뉴스판에서',
 '주요뉴스를',
 '볼',
 '수',
 '있습니다',
 '보러가기',
 '매일경제',
 '언론사',
 '구독',
 '해지되었습니다',
 'PICK',
 '안내',
 '언론사가',
 '주요기사로선정한',
 '기사입니다',
 '언론사별',
 '바로가기',
 '닫기',
 '외출',
 '망설여지네',
 '천둥',
 '번개',
 '동반',
 '수도권',
 '최대',
 '100',
 '물폭탄',
 '입력2024',
 '07',
 '08',
 '오전',
 '11',
 '16',
 '기사원문',
 '이가람',
 '기자',
 '이가람',
 '기자',
 '구독',
 '구독중',
 '구']

In [20]:
" ".join(re.findall("[가-힣a-zA-Z0-9_]+", docs[0].page_content[:500]))

'외출 망설여지네 천둥 번개 동반 수도권 최대 100 물폭탄 본문 바로가기 NAVER 뉴스 연예 스포츠 날씨 프리미엄 검색 언론사별 정치 경제 사회 생활 문화 IT 과학 세계 랭킹 신문보기 오피니언 TV 팩트체크 알고리즘 안내 정정보도 모음 구독 매일경제 언론사 구독되었습니다 메인 뉴스판에서 주요뉴스를 볼 수 있습니다 보러가기 매일경제 언론사 구독 해지되었습니다 PICK 안내 언론사가 주요기사로선정한 기사입니다 언론사별 바로가기 닫기 외출 망설여지네 천둥 번개 동반 수도권 최대 100 물폭탄 입력2024 07 08 오전 11 16 기사원문 이가람 기자 이가람 기자 구독 구독중 구'

#### PDF 파일의 텍스트 데이터 가져오기

In [21]:
from langchain.document_loaders import PyPDFLoader

In [22]:
loader = PyPDFLoader("../data/기술보증기금과 한국경제.pdf")

docs = loader.load_and_split()

print(len(docs))
docs
# 여러 페이지가 있는 파일의 경우 load_and_split() 함수를 그대로 사용하면 페이지 별로 Document 객체로 반환

5


[Document(page_content='페이지  1 / 5 \n \n기술보증기금과  한국경제  \n \nI. 기술보증기금  개요  \n1. 설립근거  : 기술보증기금법  \n \n- 설립목적 (존재이유 ) \n✓ 기술보증기금을  설립하여  기술보증제도를  정착·발전시킴으로써  신기술사업에  대한 자금의  \n공급을  원활하게  하고 나아가  국민경제의  발전에  이바지함을  목적으로  함(기술보증기금법 1\n조) \n✓ 설립 : 담보능력이  미약한  기업의  채무를  보증하게  하여 기업에 대한 자금 융통을  원활하게  하기 \n위하여  기술보증기금을  설립(법 12조) \n✓ 기금의  재원 : 정부 출연금의  예산은  중소벤처기업부  소관으로  함 \n \n☞ 기술보증기금은   \n✓ 기술력은  우수하지만  담보 부족한  중소기업의  \n✓ 기술성과  사업성  평가를  통해 기술보증을  지원하며 , \n✓ 기술평가 , 벤처이노비즈기업  인증, 중소기업  창업지원  등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증  신기술사업자가  부담하는  금전 채무 보증.(신용부족 -담보부족  해결) \n \n금융회사  등으로부터  자금 대출을 받음으로써  금융회사  등에 대하여  부담하는  금전 채무를 \n기술보증기금이  기술보증서로  보증 \n 신기술사업자  - 기술을  개발하거나  이를 응용하여  사업화하는  중소기업 (「중소기업기본법」에  \n따른 중소기업 ) 및 대통령령으로  정하는  기업 \n- "기업"이란 사업을  하는 개인 및 법인 \n신용보증  상시 사용하는  종업원이  1천명 이하이고  총자산액이  1천억원  이하인  기업이  부담하는  금전\n채무 보증. \n기술평가  해당 기술과  관련된  기술성·시장성·사업 타당성 등을 종합적으로  평가하여  \n금액·등급·의견  또는 점수 등으로  표시하는  것을 말한다  \n* 기금은  기술평가의  객관성  및 공정성  등을 확보하기  위하여  기술평가의  \n기준·절차·방법·종

In [23]:
pdf_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 50,
    separators = ["\n\n", "\n", " "]
)

docs_split = loader.load_and_split(text_splitter = pdf_splitter)
print(len(docs_split))
docs_split

26


[Document(page_content='페이지  1 / 5 \n \n기술보증기금과  한국경제  \n \nI. 기술보증기금  개요  \n1. 설립근거  : 기술보증기금법  \n \n- 설립목적 (존재이유 ) \n✓ 기술보증기금을  설립하여  기술보증제도를  정착·발전시킴으로써  신기술사업에  대한 자금의  \n공급을  원활하게  하고 나아가  국민경제의  발전에  이바지함을  목적으로  함(기술보증기금법 1\n조) \n✓ 설립 : 담보능력이  미약한  기업의  채무를  보증하게  하여 기업에 대한 자금 융통을  원활하게  하기 \n위하여  기술보증기금을  설립(법 12조)', metadata={'source': '../data/기술보증기금과 한국경제.pdf', 'page': 0}),
 Document(page_content='위하여  기술보증기금을  설립(법 12조) \n✓ 기금의  재원 : 정부 출연금의  예산은  중소벤처기업부  소관으로  함 \n \n☞ 기술보증기금은   \n✓ 기술력은  우수하지만  담보 부족한  중소기업의  \n✓ 기술성과  사업성  평가를  통해 기술보증을  지원하며 , \n✓ 기술평가 , 벤처이노비즈기업  인증, 중소기업  창업지원  등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증  신기술사업자가  부담하는  금전 채무 보증.(신용부족 -담보부족  해결)', metadata={'source': '../data/기술보증기금과 한국경제.pdf', 'page': 0}),
 Document(page_content='기술보증  신기술사업자가  부담하는  금전 채무 보증.(신용부족 -담보부족  해결) \n \n금융회사  등으로부터  자금 대출을 받음으로써  금융회사  등에 대하여  부담하는  금전 채무를 \n기술보증기금이  기술보증서로  보증 \n 신기술사업자  - 기술을  개발하거나  이를 응용하여  사업화하는  중소기업 (「중소기업기본법」에  \n따른 중소기업 ) 및 대통령령으로  정하는  기업 \n- "